In [ ]:
# Authors:

# Optimisation and Machine Learning for Process Systems Engineering:
# https://www.imperial.ac.uk/optimisation-and-machine-learning-for-process-engineering/about-us/



The first part of this notebook is a recap from the course. There is a practical section after this recap.

You will find <font color='blue'>text in blue for a few simple planned **coding tasks** that you can implement.</font>

Within the code cell, we suggest your implementation to be between the line:

`============= Start of your code =============`

and the line:

`============= End of your code =============` **bold text**

# Problem description

Summary of the general anomaly detection problem

* Anomaly data is scarce, therefore it is considered that we can only use normal operation data for training. The few instances of fault behavior we have are left for validation purposes
* Therefore, we face a semisupervised problem. We know the labels of normal instances only and we only feed the training with them. The distinction between both classes must be made *after* and not *during* training through some kind of threshold tuning.
* Typical methods used range from statistical, neural networks, distance-based (<font color='red'>summarize here more from time series analysis</font>)

# Warm-up: single variable example

In this example, we are generating temperature data from a reactor. We have two sets of data:

- The training set, which we are certain contains normal operation conditions (NOC) only
- The cross-validation set, which contains a mix of both normal and anomaly conditions

We will use statistical methods to try to predict the state of the system in the validation dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
np.random.seed(41)

# Generate normal operation conditions (NOC) data
mean    = 400
std = 3
n_noc = 500

X_train = np.random.normal(mean, std, n_noc)

# Time-series plot
x = np.linspace(X_train.min() - 2, X_train.max() + 2, 1000)
plt.plot(X_train)
plt.title('Normal operation data against time')
plt.xlabel('Time (s)')
plt.ylabel('Reactor Temperature')
plt.show()

# Histogram
plt.hist(X_train, 20, density=True, facecolor='g', alpha=0.75, label='data')
plt.plot(x, scipy.stats.norm.pdf(x, mean, std), color='black', label='normal distribution')
plt.xlabel('Temperature')
plt.xlim(min(x), max(x))
plt.legend(loc='best')
plt.show()

We can select a specific threshold over which any value will be considered an anomaly. A typical one is $2\sigma$, which corresponds with about a 95 % confidence level

Note that, in real-life examples, we do not know the exact mean and standard deviation of our data. Therefore, the way to work with them is estimating the mean and standard deviation of our sample in the training set.

In [ ]:
mean_train = X_train.mean()
std_est = X_train.std()
th_lower = mean_train - 2 * std_est
th_higher = mean_train + 2 * std_est

In [ ]:
# Get NOC bounds
x_fill = np.linspace(th_lower, th_higher)
fig_pdf, ax_pdf = plt.subplots()
ax_pdf.plot(x, scipy.stats.norm.pdf(x, mean, std), color='black', label='normal distribution')
ax_pdf.fill_between(x_fill, scipy.stats.norm.pdf(x_fill, mean, std), color='grey')
# Get anomaly bounds
x_fault = np.linspace(min(x), th_lower)
ax_pdf.fill_between(x_fault, scipy.stats.norm.pdf(x_fault, mean, std), color='salmon')
x_fault = np.linspace(th_higher, max(x))
ax_pdf.fill_between(x_fault, scipy.stats.norm.pdf(x_fault, mean, std), color='salmon')
ax_pdf.set_xlim(min(x), max(x))
ax_pdf.set_ylim([0, None])
ax_pdf.set_xlabel('Temperature')

In [ ]:
# Generate anomaly data with drift
n_noc = 100
n_fault_slice = 50  # 50 slices of 10 points each
n_slice = 10
changeRate = 1.01
X_test = np.random.normal(mean, std, n_noc)

poisson_variable = np.random.poisson(0.05, n_fault_slice);
mu_fault = mean
sigma_fault = std
for i in range(0, n_fault_slice):
    mu_fault = mu_fault * (changeRate ** poisson_variable[i])
    sigma_fault = sigma_fault * (changeRate ** poisson_variable[i])
    next_fault_data = np.random.normal(mu_fault, sigma_fault, n_slice)
    X_test = np.concatenate((X_test, next_fault_data))

# Time series plot
x_fault = np.arange(1, len(X_test) + 1)
plt.plot(x_fault, X_test)
plt.axvline(n_noc, color='salmon', label='fault starts')

plt.plot([x_fault[0], x_fault[-1]], [th_higher, th_higher], 'r--')
plt.plot([x_fault[0], x_fault[-1]], [th_lower, th_lower], 'r--')

plt.title('Anomaly generation against time')
plt.xlabel('Time (s)')
plt.ylabel('Reactor Temperature')
plt.legend(loc='best')
plt.show()

# Histogram
ax_pdf.hist(X_test, 20, density=True, facecolor='g', alpha=0.75, label='anomaly data')
ax_pdf.legend(loc='best')
fig_pdf

Let's obtain a metric for how well does this anomaly detector worked.

In [ ]:
# First we get the actual labels of our cross-validation set
# We know that the first 100 instances are normal, while the fault starts
#  from that point onwards
y_true = np.concatenate((
    np.zeros((n_noc,)),
    np.ones((n_fault_slice * n_slice))
))
y_pred = (X_test > th_higher) | (X_test < th_lower)

# Plot our prediction
plt.plot(x_fault[y_pred == 0], X_test[y_pred == 0], 'b.', label='predicted as normal')
plt.plot(x_fault[y_pred == 1], X_test[y_pred == 1], 'rx', label='predicted as anomaly')
plt.title('Anomalies detected over time')
plt.xlabel('Time (s)')
plt.ylabel('Reactor Temperature')
plt.legend(loc='best')

# Use the accuracy_score function from skit-learn
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true, y_pred)
print(f"The obtained accuracy is {accuracy * 100:0.2f} %")

Note that due to our threshold, a few false positives take place. This is a common trade-off in anomaly detection:
- **It is usually difficult to avoid both false positives and false negatives** at the same time
- The determination of the threshold is an important task that the engineer must take into account to give more weight to one or another.
- This depends heavily on the application: e.g. in medical applications, false positives are highly discouraged, while spam detection might be more indulgent in that sense

Also note that the fault takes quite a while to manifest, from it start at $t=100$ min to the first visually noticeable drift at aroun $t=300$ min. More advanced methods might help us anticipate this anomaly but, without more variables showing more hints, it can be impossible to increase our accuracy.

<font color = blue>Now, you will need to write code to detect the anomalies in this dataset fitting the data to a normal distribution and reporting the outliers using the overall accuracty as a metric.<\font>

To determine a threshold to distinguish normal from anomalous instances, use three times the standard deviation ($3\sigma$)

In [ ]:
# ============= Start of your code ============= #


# =============  End of your code  ============= #

<font color='blue'>Now tune this threshold using a numerical method to get a specific performance </font> (<font color='red'>not useful now since it can be demonstrated that a specific sigma leaves a specific % out of "normality". BUT will be useful for Autoencoders and PCA</font>) ?

## [Optional] repeat stacking time windows

# Multivariate example

Now, consider a more complex case where we have 10 variables that conform a dataset representative of the state of the system.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.metrics import accuracy_score
np.random.seed(41)

In [ ]:
def generate_multivariate_normal_dataset(mean, covariance, n, perc_noc, changing_rate, rng=None, shuffle=False):
  n_noc = int (n * perc_noc)
  n_fault = n - n_noc

  x_noc = np.zeros((n_noc, 10))
  x_fault = np.zeros((n_fault, 10))

  # Generate data under normal conditions
  x_noc = rng.multivariate_normal(mean=mean, cov=covariance, size=n_noc)
  label_noc = np.zeros(n_noc)
  label_noc = label_noc.reshape((len(label_noc), 1))
  data_noc = np.concatenate((x_noc, label_noc), axis=1)

  # Generate data include fault
  poisson_generator = rng.poisson(0.2, n_fault)
  label_poisson = np.ones(n_fault)
  for j in range(n_fault):
    if poisson_generator[j] != 0:
        label_poisson[0:j+1] = 0
        break

  # Depends on whether mean or cov is changing
  mean_fault = mean
  cov_fault = covariance

  for i in range(x_fault.shape[0]):
    if poisson_generator[i] != 0:
        cov_fault[0,1] *= (1+changing_rate)**(poisson_generator[i]) # make it also accumulated(to detect as soon as possible)
        cov_fault[1,0] *= (1+changing_rate)**(poisson_generator[i])
    instance = rng.multivariate_normal(mean=mean_fault, cov=cov_fault, size=1)
    x_fault[i] = instance

  label_poisson = label_poisson.reshape((len(label_poisson), 1))
  data_fault = np.concatenate((x_fault, label_poisson), axis=1)

  data_np = np.concatenate((data_noc, data_fault), axis=0)
  data = pd.DataFrame(data_np, columns=['x1', 'x2', 'x3', 'x4','x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'label'])

  if shuffle == True:
    data.sample(frac=1).reset_index(drop=True)

  return data, data_np

In [ ]:
# Generate data
n_samples = 500
perc_noc_train = 1   # % NOC instances in the training set (100 %)
perc_noc_test = 0.2  # % NOC instances in the test set (100 %)
changing_rate = 0.15
base_rng      = np.random.default_rng(42)

# Generate means
mean = base_rng.integers(low=1, high=10, size=10)
print(mean)

# Generate covariance
covariance = np.array([[2.0, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5],
                      [-0.5, 2.0, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5],
                      [0.5, -0.5, 2.0, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5],
                      [-0.5, 0.5, -0.5, 2.0, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5],
                      [0.5, -0.5, 0.5, -0.5, 2.0, -0.5, 0.5, -0.5, 0.5, -0.5],
                      [-0.5, 0.5, -0.5, 0.5, -0.5, 2.0, -0.5, 0.5, -0.5, 0.5],
                      [0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 2.0, -0.5, 0.5, -0.5],
                      [-0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 2.0, -0.5, 0.5],
                      [0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 2.0, -0.5],
                      [-0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, 2.0]])
print(covariance)


data_train, data_train_np = generate_multivariate_normal_dataset(mean, covariance, n_samples, perc_noc_train, changing_rate, rng=base_rng, shuffle=False)
data_test, data_test_np = generate_multivariate_normal_dataset(mean, covariance, n_samples, perc_noc_test, changing_rate, rng=base_rng, shuffle=False)
X_train = data_train.drop(['label'], axis=1).to_numpy()
y_train = data_train['label'].to_numpy()
X_test = data_test.drop(['label'], axis=1).to_numpy()
y_test = data_test['label'].to_numpy()

# The features of the dataset correspond to the columns of X_train
n_features = X_train.shape[1]

In [ ]:
X_train

In this case, the dataset is completly random, and disturbances are introduced in the covariance between the first and second variable, which starts suffering a drift based on a Poisson disturbance like in the single-variable example

In [ ]:
plt.plot(data_test.iloc[:200,0])
plt.plot(data_train.iloc[:200,0])

## Probabilistic approach


We estimate the detection thresholds for each variable separately. For this, we use the probability density function of the normal distribution

$f(x_i|\mu, \sigma^2) = \frac{1}{{\sqrt{2 \pi \sigma^2}}} e^{-\frac{{(x - \mu)^2}}{{2 \sigma^2}}}$

XXXXXXXXXXXXXXXX Note that the probability density function is not exactly yielding probabilites, but a measure of the density of the random distribution at a specific point. however, it serves as a way to determine how frequent a range of values are  

In [ ]:
def probability_density(X, mean, var):
    """
    Calculate the probability density of a series of instances in X given the
    multivariate mean and covariance matrix.
    """
    n = len(mean)
    if var.ndim == 1:
        var = np.diag(var)
    X = X - mean
    p = (2 * np.pi)**(- n/2) * np.linalg.det(var)**(-0.5) * \
        np.exp(-0.5 * np.sum(np.matmul(X, np.linalg.pinv(var)) * X, axis=1))
    return p

Now we estimate the means and covariance of our system using the training set

In [ ]:
# Estimate Gaussian distribution
mean_train = np.mean(X_train, axis=0)
cov_train = np.cov(X_train, rowvar=False)  # By default it treats each row as a variable


Fit the threshold using the training set

In [ ]:
def get_threshold(scores, y_true, target_acc=0.95, positive_lower=True, n_samples=1e4):
    """ 
    Obtain threshold by searching for the closest accuracy to target_acc via 
    brute force.
    ----------
    Parameters
    ----------
    positive_lower: if True, poisitive instances are predicted when the score
        is lower than the threshold. If False, positive instances are predicted
        for socres higher than the threshold
    """
    # Get extremes of scores
    scores_range = np.linspace(scores.min(), scores.max(), int(n_samples))
    # Loop them until finding the closer value to 95 % accuracy (brute force)
    error_old = 1
    accs = {}
    for score_th in scores_range:
        # Obtain predictions given the current threshold
        if positive_lower:
            y_pred = scores < score_th
        else:
            y_pred = scores > score_th
        # Calculate accuracy and measure the error with respect target_acc
        acc = accuracy_score(y_true, y_pred)
        accs[score_th] = acc
        error = (target_acc - acc) ** 2
        print(f"Obtained accuracy {acc}, error {error}" 
              f" with threshold = {score_th}")
        if error < error_old:
            threshold = score_th 
            error_old = error
        elif error > error_old:
            # Avoid continuing since no better error will be obtained
            break
    plt.plot(pd.Series(accs), label='evolution of accuracy')
    plt.vlines(threshold, plt.gca().get_ylim()[0], 1, color='red', label='final threshold')
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    return threshold

In [ ]:
# Get probabilities and threshold
probs = probability_density(X_train, mean_train, cov_train)
threshold = get_threshold(probs, y_train, positive_lower=True, n_samples=1e5)

Check we obtain the desired accuracy

In [ ]:
y_train_pred = probability_density(X_train, mean_train, cov_train) < threshold
print(f'We should have {int(len(X_train) * 0.05)} false positives.')
print(f'We have {sum(y_train_pred)} false positives')

Now that we have our threshold, we can evaluate the test set. Remember we use the fitted mean and variance from the training set, we do not obtain these from the test set.

In [ ]:
p_test = probability_density(X_test, mean_train, cov_train) 
y_test_pred = p_test < threshold
acc_test = accuracy_score(y_test, y_test_pred)
print(f'The obtained accuracy is {acc_test *100} %')

## Proximity-based approach: Mahalanobis distance

In [ ]:
# Compute centroid of the training set, corresponding to the estimated mean of each feature
means = X_train.mean(axis=0)
print(means)
cov = np.cov(X_train, rowvar=False)
inv_cov = np.linalg.inv(cov)
print(inv_cov)

In [ ]:
from scipy.spatial.distance import mahalanobis

def compute_distances(X, means, inv_cov):
    """
    Compute the Mahalanobis distance between each normal data point and the 
    cluster centers.
    """
    distances_train = []
    for i in range(X.shape[0]):
        distances_train.append(mahalanobis(X[i, :], means, inv_cov))
    return np.array(distances_train)

In [ ]:
distances_train = compute_distances(X_train, means, inv_cov)

In [ ]:
plt.hist(distances_train, 20, density=True, facecolor='g', alpha=0.75, label='data')

In [ ]:
th_mahalanobis = get_threshold(distances_train, y_train, positive_lower=False)

In [ ]:
y_pred_train = distances_train > th_mahalanobis
print(f'We should have {int(len(X_train) * 0.05)} false positives.')
print(f'We have {sum(y_pred_train)} false positives')

In [ ]:
distances_test = compute_distances(X_test, means, inv_cov)
y_pred_test = distances_test > th_mahalanobis
acc_test = accuracy_score(y_test, y_pred_test)
print(f'The obtained accuracy is {acc_test *100} %')

# Neural networks: autoencoders

Neural networks are usually trained in a supervised manner. To be able to use them in a semisupervised problem, we cannot use the typical feedforward neural network. Fo this, the self-associative neural networks, also called autoencoders, come to help. Their name is self-explanatory: the goal of autoencorders is to try to reproduce the input in the output of the neural network. This may look like a trivial task but the trick here is that there is a bottleneck in the architecture of the network that causes a loss of information.

The training phase tries to make the reconstruction with the lowest possible loss despite the bottleneck. This way it is able to find the most important features and discard noise that is useless to reconstruct the input. Its usefulness in anomaly detection comes with that, if trained with normal operation data and learns to identify and reconstruct it correctly, when a abnormal event occurs, the reconstruction will show an important bias compared with nomal operation data.

Therefore, they do not use labels to train the network. However, we need to know that all the instances fed during training belong to the same class (usually the *normal operation* class), therefore the *semisupervised* surname of it.

We will work with the mnist dataset. For this, we will try to identify as an anomaly every number that is not a "5"

![title](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

First, we will load the dataset from the scikit-learn builtin datasets

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

In [ ]:
def plot_mnist(X, y, n=3, title=''):
    """ Distribute plots in rows of 4. """
    from math import ceil
    nrows = ceil(n / 4)
    ncols = 4
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 10))
    i = 0
    for image, label in zip(X, y):
        #np.unravel_index(i, axes.shape)
        ax = axes.flatten()[i]
        ax.set_axis_off()
        image = image.reshape(8, 8)
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
        if not label:
            ax.set_title(f"Label: {bool(label)}", fontweight="bold")
        else:
            ax.set_title(f"Label label: {bool(label)}")
        i += 1
        if i >= n:
            break
    fig.suptitle(title)


def get_mnist_5(split=False):
    # Load data
    digits = datasets.load_digits()

    # Get input and output data
    X = digits.data
    labels = digits.target
    y = ~ (labels == 5)

    if split:
        # Split data into 50% train and 50% test subsets
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, shuffle=True
        )
        return X_train, X_test, y_train, y_test
    else:
        return X, y
    
    
X_train, X_test, y_train, y_test = get_mnist_5(split=True)
plot_mnist(X_train, y_train, n=12)
print("Five's are represented as False, while the rest is considered True")

# Keep only 5's for the training set
X_train = X_train[y_train == False]
y_train = y_train[y_train == False]
plot_mnist(X_train, y_train, n=12)

Each instance of the training set consists of 64 elements that represent the 8-by-8 greyscale pixel image of a handwritten number. Each pixel value ranges from 0 to 16, from white to black

In [ ]:
print(X_train[1, :], y_train[0])

Before working with the data, we need to standardize the pixel intensisty values

In [ ]:
X_train_norm = X_train / 16
X_test_norm = X_test / 16

Now we have our dataset, we can set up our autoencoder neural network.

First, set up the architecture:

In [ ]:
# Shape of input and latent variable
n_input = X_train.shape[1]

# Encoder structure
n_encoder1 = 10
# n_encoder2 = 10
n_latent = 2

# Decoder structure
# n_decoder2 = 10
n_decoder1 = 10

hidden_layer_sizes = (n_encoder1, n_latent, n_decoder1)

Initialize the model

In [ ]:
model = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes, 
    activation='tanh', 
    solver='adam', 
    batch_size='auto', 
    learning_rate_init=0.001,
    max_iter=1000,
    random_state=None,
    tol=0.0000001,
    verbose=True,
    alpha=0.0001
)
model

Now we can fit the model. Bear in mind that we want the model to try to obtain a target output as similar as possible to the input.

In [ ]:
model.fit(X_train_norm, X_train_norm)

In [ ]:
plt.plot(model.loss_curve_)

Check how reconstructions go for the trained model.

In [ ]:
def compare_inout_mnist(image_in, image_out):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    axes[0].imshow(image_in, cmap=plt.cm.gray_r, interpolation="nearest")
    axes[0].set_title("Input")
    axes[1].imshow(image_out, cmap=plt.cm.gray_r, interpolation="nearest")
    axes[1].set_title("Output")
    plt.show()
    

In [ ]:
# Plot 5
input_image = X_train_norm[0, :].reshape((8,  8))
# Perform reconstruction through autoencoder
reconstructed_image = model.predict(X_train_norm[0:0 + 1, :])
reconstructed_image = reconstructed_image.reshape((8, 8))
compare_inout_mnist(input_image, reconstructed_image)

In [ ]:
X_test_norm[:, :].max()

In [ ]:
# Plot other numbers in the test set
reconstructed_images = []
for i in range(1, 10):
    input_image = X_test_norm[i, :].reshape((8,  8))
    # Perform reconstruction through autoencoder
    reconstructed_image = model.predict(X_test_norm[i:i + 1, :])
    print(reconstructed_image.max(), reconstructed_image.min())
    reconstructed_image = reconstructed_image.reshape((8, 8))
    compare_inout_mnist(input_image, reconstructed_image)
    reconstructed_images.append(reconstructed_image)

**Now explain how to measure the reconstruction error**